<a href="https://colab.research.google.com/github/hoyathali/ParrotTransformation/blob/main/parrotmodel_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch

In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
raw_data_path='output.csv'
df= pd.read_csv(raw_data_path,chunksize=50000,header=None)
transposed_data=df
for d in df:
    transposed_data=d.transpose()

print(transposed_data.shape)


(1570, 16384)


In [ ]:
no_of_features=64*64*3
no_of_outputs=64*64

X=transposed_data.iloc[:,:no_of_features]
y=transposed_data.iloc[:, -no_of_outputs:]

print(X.shape)
print(y.shape)

(1570, 12288)
(1570, 4096)


In [ ]:
# Step 1: Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Normalize the data
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and validation data
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)
X_test_normalized = scaler.transform(X_test)

# Convert normalized data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)  # Assuming labels are floats

X_val_tensor = torch.tensor(X_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Step 3: Create PyTorch datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        hidden_size = (input_size // 3) * 2
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.fc3 = nn.Linear(hidden_size//2, hidden_size*2)
        self.fc4 = nn.Linear(hidden_size*2, output_size)

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)

        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)
        nn.init.xavier_uniform_(self.fc4.weight)
        nn.init.zeros_(self.fc4.bias)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x2=x+x2
        x = self.fc4(x)

        return x

input_size = 64 * 64 * 3
output_size = 64 * 64


In [ ]:
batch_size = 16  # Adjust the batch size based on your needs

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

model = SimpleNN(input_size, output_size)

# Check if GPU is available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the GPU
model.to(device)

# Move the criterion and optimizer to the GPU
criterion = nn.MSELoss().to(device)
#optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.001)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# Early stopping parameters
early_stopping_patience = 5
best_val_loss = float('inf')
counter = 0

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move inputs and targets to GPU
        inputs, targets = inputs.to(device), targets.to(device)

        # Flatten the inputs if needed
        inputs = inputs.view(inputs.size(0), -1)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, targets)

        # L2 regularization term
        l2_reg = torch.tensor(0.).to(device)
        for param in model.parameters():
            l2_reg += torch.norm(param, p=2)

        loss += 0.001 * l2_reg

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    # Print the average training loss for the epoch
    average_train_loss = running_train_loss / len(train_loader)
    print(f'Training Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0

    with torch.no_grad():
        for batch_idx, (val_inputs, val_targets) in enumerate(val_loader):
            # Move validation inputs and targets to GPU
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
            val_inputs = val_inputs.view(val_inputs.size(0), -1)

            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_targets)
            running_val_loss += val_loss.item()

    # Print the average validation loss for the epoch
    average_val_loss = running_val_loss / len(val_loader)
    print(f'Validation Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_val_loss:.4f}')

    # Early stopping check
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        counter = 0
    else:
        counter += 1
        print(f'Validation loss did not improve. Patience left: {early_stopping_patience - counter}/{early_stopping_patience}')
        if counter >= early_stopping_patience:
            print(f'Early stopping after {epoch + 1 - early_stopping_patience} epochs without improvement.')
            break

print('Training finished!')

Training Epoch [1/100], Average Loss: 10324.1509
Validation Epoch [1/100], Average Loss: 5548.7564
Training Epoch [2/100], Average Loss: 6595.1299
Validation Epoch [2/100], Average Loss: 5893.1281
Validation loss did not improve. Patience left: 4/5
Training Epoch [3/100], Average Loss: 4123.7117
Validation Epoch [3/100], Average Loss: 4353.1885
Training Epoch [4/100], Average Loss: 3878.1450
Validation Epoch [4/100], Average Loss: 3249.6125
Training Epoch [5/100], Average Loss: 2549.5923
Validation Epoch [5/100], Average Loss: 3592.8558
Validation loss did not improve. Patience left: 4/5
Training Epoch [6/100], Average Loss: 2457.6282
Validation Epoch [6/100], Average Loss: 2754.0172
Training Epoch [7/100], Average Loss: 2224.9350
Validation Epoch [7/100], Average Loss: 2720.8803
Training Epoch [8/100], Average Loss: 2195.5124
Validation Epoch [8/100], Average Loss: 2720.8695
Training Epoch [9/100], Average Loss: 1872.2095
Validation Epoch [9/100], Average Loss: 2425.5253
Training Epoc

In [ ]:
torch.save(model,'parrot.pth')

checkpoint = {
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': average_val_loss,
    'model_architecture': SimpleNN(input_size, output_size)
}
torch.save(checkpoint, 'parrot.pth')

In [ ]:
from PIL import Image
import numpy as np
import os
from torchvision import transforms

img = Image.open("3.jpg")

img = img.resize((64,64))
img.save("3.jpg")

transform = transforms.Compose([
transforms.ToTensor(),
    ])

img_tensor = transform(img).unsqueeze(0)
img_tensor = img_tensor.view(1, -1)

input_size = 64 *64*3  # Replace with the actual input size
output_size = 64*64  # Replace with the actual output size

# Load the model architecture and state dictionary
checkpoint = torch.load('parrot.pth')
model_architecture = SimpleNN(input_size, output_size)
model_architecture.load_state_dict(checkpoint['model_state_dict'])
model_architecture.eval()

with torch.no_grad():
    output = model_architecture(img_tensor)

output = output.view(1, 64, 64)
output = Image.fromarray(output.squeeze().numpy().astype('uint8'))

output.save("3_output.jpg")
